In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_884dfcada24c4dc19214c13c883622bd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XIhgF3UT3z3xstsQn3B9Hl8wVDRhftIFjic6oOBlqBbP',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

body = client_884dfcada24c4dc19214c13c883622bd.get_object(Bucket='b7iris-donotdelete-pr-5fopib9lvera8t',Key='iris.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,sepallength(cm),sepalwidth(cm),petallength(cm),petalwidth(cm),class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
import pandas as pd

In [ ]:
X = df.iloc[:, :-1]
y = df['class']

In [ ]:
X.head()

,sepallength(cm),sepalwidth(cm),petallength(cm),petalwidth(cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [ ]:
y.head()

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: class, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=50)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=40)
model.fit(x_train, y_train)

RandomForestClassifier(random_state=40)

In [ ]:
from sklearn.metrics import accuracy_score
y_predict = model.predict(x_test)
accuracy_score(y_test, y_predict)

0.9666666666666667

In [ ]:
pd.crosstab(y_test, y_predict)

col_0,Iris-setosa,Iris-versicolor,Iris-virginica
class,,,
Iris-setosa,9,0,0
Iris-versicolor,0,11,1
Iris-virginica,0,0,9


## IBM -Deployment

In [ ]:
!pip install -U ibm-watson-machine-learning

     |████████████████████████████████| 1.8 MB 15.1 MB/s eta 0:00:01███████████████████████████▎    | 1.5 MB 15.1 MB/s eta 0:00:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.238
    Uninstalling ibm-watson-machine-learning-1.0.238:
      Successfully uninstalled ibm-watson-machine-learning-1.0.238


In [ ]:
from ibm_watson_machine_learning import APIClient
import json

## Authenticate and set space

In [ ]:
wml_credentials = {
    "apikey":"kdw7pwytkDuH1FTfuY6WS36PoxVezw3h51gPGuIzYP9D",
    "url":"https://us-south.ml.cloud.ibm.com"
}

In [ ]:
wml_client = APIClient(wml_credentials)

In [ ]:
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -------  ------------------------
ID                                    NAME     CREATED
5581c26a-8159-46ab-9c2d-2da5055bc1e2  iris-B7  2022-10-17T09:05:53.347Z
------------------------------------  -------  ------------------------


In [ ]:
SPACE_ID= "5581c26a-8159-46ab-9c2d-2da5055bc1e2"

In [ ]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [ ]:
wml_client.software_specifications.list(100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

## Save and Deploy the model

In [ ]:
import sklearn
sklearn.__version__

'1.0.2'

In [ ]:
MODEL_NAME = 'iris-B7'
DEPLOYMENT_NAME = 'iris-B7'
DEMO_MODEL = model

In [ ]:
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [ ]:
# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [ ]:
#Save model

model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=x_train, 
    training_target=y_train
)

In [ ]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'class',
  'schemas': {'input': [{'fields': [{'name': 'sepallength(cm)',
       'type': 'float64'},
      {'name': 'sepalwidth(cm)', 'type': 'float64'},
      {'name': 'petallength(cm)', 'type': 'float64'},
      {'name': 'petalwidth(cm)', 'type': 'float64'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-10-17T09:21:33.324Z',
  'id': '60c52e65-0cb3-4d96-b224-244582335131',
  'modified_at': '2022-10-17T09:21:36.772Z',
  'name': 'iris-B7',
  'owner': 'IBMid-6610042FA2',
  'resource_key': 'b1f43d6e-933b-4056-aae8-e8c635d31e22',
  'space_id': '5581c26a-8159-46ab-9c2d-2da5055bc1e2'},
 'system': {'warnings': []}}

In [ ]:
model_id = wml_client.repository.get_model_id(model_details)
model_id

'60c52e65-0cb3-4d96-b224-244582335131'

In [ ]:
# Set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [ ]:
# Deploy
deployment = wml_client.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)



#######################################################################################

Synchronous deployment creation for uid: '60c52e65-0cb3-4d96-b224-244582335131' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='6d428612-296a-478d-9707-a0877a01672e'
------------------------------------------------------------------------------------------------




In [ ]:
## 3.20